<a href="https://colab.research.google.com/github/kabilan942/Natural-Language-Processing/blob/main/Legal%20Document%20Summarization/Extractive_Summarization_(Legal_Domain).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Modules Required

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install rouge
!pip install rouge_score
!pip install datasets evaluate transformers[sentencepiece]
!pip install nltk
!pip install datasets
!pip install -U sentence-transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
nltk.download('punkt')
from rouge import Rouge
from datasets import Dataset, DatasetDict, load_dataset, load_metric
from nltk.tokenize import sent_tokenize
import pandas as pd
import numpy as np
import glob
from tqdm import tqdm
from nltk import tokenize
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import torch.nn.functional as F
import torch
import json
import random
import os
import sys
sys.path.insert(0, '../')
from IPython.display import display, HTML
#from transformers import LEDTokenizer, LEDForConditionalGeneration
#from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import evaluate
rouge_score = evaluate.load('rouge')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!pip install tensorfl

ERROR: Could not find a version that satisfies the requirement tensorfl (from versions: none)
ERROR: No matching distribution found for tensorfl


In [ ]:
from transformers import BertTokenizer, TFBertModel

In [ ]:
model = TFBertModel.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.8.4
aiosignal                        1.3.1
alabaster                        0.7.13
albumentations                   1.2.1
altair                           4.2.2
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      21.3.0
argon2-cffi-bindings             21.2.0
array-record                     0.4.0
arviz                            0.15.1
astropy                          5.2.2
astunparse                       1.6.3
async-timeout                    4.0.2
attrs                            23.1.0
audioread                        3.0.0
autograd                         1.6.2
Babel                            2.12.1
backcall                         0.2.0
beautifulsoup4                   4.11.2
bleach                           6.0.0
blis                             0.7.9


In [ ]:
import torch

cuda_version = torch.version.cuda
print("CUDA Version:", cuda_version)

CUDA Version: 11.8


## Chunking - using similarity measures


In [ ]:
from rouge_score import rouge_scorer
from evaluate import load

def saliency_score_rougef(doc, summary, alpha=0.5):
  # takes in two inputs and returns the saliency score
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)
  scores = scorer.score(doc, summary)
  R1 = scores['rouge1'][2]
  R2 = scores['rouge2'][2]

  return alpha * R1 + (1 - alpha) * R2

def saliency_score_rouger(doc, summary, alpha=0.5):
  # takes in two inputs and returns the saliency score
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)
  scores = scorer.score(doc, summary)
  R1 = scores['rouge1'][0]
  R2 = scores['rouge2'][0]

  return alpha * R1 + (1 - alpha) * R2

def calculate_bleu_score(reference, candidate):
    # Convert the reference and candidate sentences into lists of tokens
    reference_tokens = nltk.word_tokenize(reference)
    candidate_tokens = nltk.word_tokenize(candidate)

    # Calculate BLEU score using NLTK's sentence_bleu function
    smoothing_function = nltk.translate.bleu_score.SmoothingFunction().method1
    bleu_score = nltk.translate.bleu_score.sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=smoothing_function)

    return bleu_score

def saliency_score_bleu(doc, summary):
  # inputs: (doc[reference_sentence], summary[candidate_sentence])
  # output: bleu score

  return calculate_bleu_score(doc, summary)

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances, manhattan_distances, cosine_similarity

def similarity_l_l(l1, l2, metric=1):
    '''
    Function to find the most similar sentence in the document for each sentence in the summary
    input:  l1 - Summary sentences
            l2 - Document sentences
    returns a list of document sentence indexes for each sentence in the summary
    '''
    if metric==7:
      document_embeddings = sbert_model.encode(l1+l2)
      similarities=cosine_similarity(document_embeddings)
      print('Cosine similarity Roberta Sentence Embeddings')

    if metric==6:
      doc_summ_list = l1+l2
      similarities = []
      for i in doc_summ_list:
        temp = []
        for j in doc_summ_list:
          temp.append(saliency_score_bleu(i,j))
        similarities.append(temp)
      print('BLEU Score')

    if metric==5:
      doc_summ_list = l1+l2
      similarities = []
      for i in doc_summ_list:
        temp = []
        for j in doc_summ_list:
          temp.append(saliency_score_rouger(i,j))
        similarities.append(temp)
      print('ROUGE Recall')

    if metric==4:
      doc_summ_list = l1+l2
      similarities = []
      for i in doc_summ_list:
        temp = []
        for j in doc_summ_list:
          temp.append(saliency_score_rougef(i,j))
        similarities.append(temp)
      print('ROUGE f1')

    if metric==3:
      document_embeddings = sbert_model.encode(l1+l2)
      similarities=euclidean_distances(document_embeddings)
      print('Euclidean similarity')

    if metric==2:
      document_embeddings = sbert_model.encode(l1+l2)
      similarities=manhattan_distances(document_embeddings)
      print('Manhattan similarity')

    if metric==1:
      document_embeddings = sbert_model.encode(l1+l2)
      similarities=cosine_similarity(document_embeddings)
      print('Cosine similarity')

    result = []
    for i in range(len(l1)):        # loops through all sentences in summary since first l1 sentences are summary sentences
        vals = similarities[i]      # list of similarity of summary sentence i with all other l1+l2 sentences
        vals = vals[len(l1):]       # considering similarity values of ith summary sentence with all other doc sentences and leaving out the summary sentences
        idx = np.argmax(vals)       # getting the index argument of the doc sent having the highest similarity with ith summary sentence
        result.append(idx)
    return result

In [ ]:
def nest_sentencesV2(document,chunk_length):
    '''
    function to chunk a document
    input:  document           - Input document
            chunk_length        - chunk length
    output: list of chunks. Each chunk is a list of sentences.
    '''
    nested = []  # to store list of chunks of a doc
    sent = []    # to store sentences per chunk in loop
    length = 0
    for sentence in nltk.sent_tokenize(document):
        length += len(sentence.split(" "))      # adding no of words per sentence
        if length < chunk_length:
            sent.append(sentence)  # sentences appended to sent till chunk_length is reached
        else:
            nested.append(sent)                 # when word limit exceed chunk length (512 for t5) add the list of sentences (a chunk) after joining to the nested list
            sent = []                           # create a new list to store next batch fof sentences for subsequent chunk
            sent.append(sentence)
            length = 0
            length += len(sentence.split(" "))   # added to include length i=of sentence in else loop. Initially not added
    if len(sent)>0:                             # when all sentences are over and the chunk of remaining sentences are appended to sent
        nested.append(sent)
    return nested

In [ ]:
def get_chunks_data_from_docV2(doc, summ, m):
    '''
    Function to generate chunks along with their summaries
    input:  doc - legal Document
            summ - Gold standard summary
    returns a list of chunks and their summaries
    '''
    chunk_summ_word_threshold = 15
    sentence_mapping = {}
    doc_sents = split_to_sentences(doc)
    summ_sents = split_to_sentences(summ)

    result = (similarity_l_l(summ_sents,doc_sents,m)) #metric = (1,2,3,4,5,6,7) = (cosine, manhattan, euclidean, Rouge f1, Rouge Recall, BLEU, cosine with roberta sentence embeddings)

    for i in range(len(summ_sents)):
        sentence_mapping[doc_sents[result[i]]] = summ_sents[i]
    # summ_sents[i]: each sentence in the summary
    # doc_sents[result[i]]: sent in doc corresponding to the index in result that had the highest similarity with that summary sentence

    final_chunks = []
    final_summ = []
    for chunk in nest_sentencesV2(doc, 400):                   # divides the doc into chunks with word limit < 512 and looping through all such chunk in doc
        summ = ""
        for chunk_sent in chunk:
            if chunk_sent in sentence_mapping:                 # if that sentence exist in the sentence_mapping keys - True
                summ = summ + sentence_mapping[chunk_sent]     # add the summary corresponding to that doc sentence to summ
        if len(summ.split(" ")) >= chunk_summ_word_threshold:  # add the chunk of doc-summary pair if the word limit of summary is more that a particular threshold
            final_chunks.append(" ".join(chunk))
            final_summ.append(summ)
    return final_chunks, final_summ

**Change Path** - Training Data

In [ ]:
def get_root_path():
    '''
    function to get root path of dataset
    change the path variable to the path of the dataset
    '''
    path = "/content/drive/MyDrive/AbstractiveDataset"
    return path

def get_summary_data():
    '''
    function to get names, documents, and summaries
    change the path variable to the path of the dataset
    '''

    path = get_root_path() + '/judgement'
    all_files = glob.glob(path + "/*.txt")
    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f:
            p = filename.rfind("/")
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)
    path = get_root_path() + '/summary'
    all_files = glob.glob(path + "/*.txt")
    data_summary = []
    for filename in all_files:
        with open(filename, 'r') as f:
            a = f.read()
            l = len(a)
            data_summary.append(a)

    return names, data_source, data_summary

def split_to_sentences(para):
    sents = nltk.sent_tokenize(para)   # returns list of sentences from para
    return sents


In [ ]:
names, data_source, data_summary = get_summary_data()
print(len(names))
print(len(data_source))
print(len(data_summary))


1
1
1


In [ ]:
from sentence_transformers import SentenceTransformer

training_chunks = []
training_summs = []
doc_id = []
metr = 1
if metr==7:
  sbert_model = SentenceTransformer('sentence-transformers/roberta-base-nli-stsb-mean-tokens')
else:
  sbert_model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')
for i in tqdm(range(len(data_source))):
    cks, summs = get_chunks_data_from_docV2(data_source[i],data_summary[i],metr) #metric = (1,2,3,4,5,6,7) = (cosine, manhattan, euclidean, Rouge f1, Rouge Recall, BLEU, cosine with roberta sentence embeddings)
    training_chunks = training_chunks + cks
    training_summs = training_summs + summs
    for j in range(len(cks)):
      doc_id.append(names[i])
df = pd.DataFrame({'data':training_chunks,'summary':training_summs,'doc_id':doc_id})
df.to_excel('chunked_mcs.xlsx')


Hello


100%|██████████| 1/1 [00:00<00:00,  1.64it/s]

Cosine similarity Roberta Sentence Embeddings


## Fine-tuning T5



In [ ]:
filepath = 'FD_MCS_512_3_train.xlsx'
df = pd.read_excel(filepath,usecols='B,C')

In [ ]:
df.shape

(22, 2)

In [ ]:
df.head()

,data,summary
0,Civil Appeal No.94 1949. 107 834 Appeal judgme...,The charge created respect municipal property ...
1,"Accordingly, reasons indicated judgment, petit...",Romesh Thappar vs The State ( [ 1950 ] S.C.R.B...
2,"In construing Act, aim purpose, Act declared i...","The expression `` public safety '', result lon..."
3,XXXVII 1950. Application article 32 Constituti...,"Neither section ( 3 ) section ( 6 ) section 4,..."
4,The law providing reasonable restrictions exer...,dissenting ) ( ) unreasonable provision contai...


In [ ]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(df, test_size=0.20,shuffle=False)

In [ ]:
train_df.shape,valid_df.shape

((17, 2), (5, 2))

In [ ]:
valid_df.reset_index(inplace=True)
valid_df.drop('index',inplace=True,axis=1)

In [ ]:
valid_df.head()

,data,summary
0,"By paragraph 12, village Maruthanthanallur Gna...","5,000 A, daughter N. '' Later, provisions wish..."
1,The testator mind different situations arise c...,5 intended apply case contingency village pass...
2,No. 37 1950. Appeal judgment Bombay High Court...,An application execution decree made expiry 12...
3,His darkhast attachment 856 present decree 3rd...,The decree holder contended judgment debtor ``...
4,The decree holder taking steps decree alive ci...,The decree holder appealed contending time Sup...


In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(valid_df)

In [ ]:
train_dataset

Dataset({
    features: ['data', 'summary', '__index_level_0__'],
    num_rows: 17
})

In [ ]:
datasets = DatasetDict()
datasets['train'] = train_dataset.remove_columns('__index_level_0__')
datasets['valid'] = val_dataset

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['data', 'summary'],
        num_rows: 17
    })
    valid: Dataset({
        features: ['data', 'summary'],
        num_rows: 5
    })
})

In [ ]:
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
max_input_length = 512
max_target_length = 256

def preprocess_function(examples):
  # Reviews are first tokenized
  model_inputs = tokenizer(examples['data'], truncation=True, max_length=max_input_length)
  # tokenizer for the target (highlights)
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples['summary'], truncation=True, max_length=max_target_length)
  # The inputs are first encoded, and then the labels ('highlights') are encoded as a separate column
  model_inputs['labels'] = labels['input_ids']
  return model_inputs

In [ ]:
tokenized_dataset = datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/17 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['data', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 17
    })
    valid: Dataset({
        features: ['data', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5
    })
})

In [ ]:
from transformers import AutoModelForSeq2SeqLM

# Seq2SeqLM: sequence-to-sequence language modeling head

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")  # model_checkpoint = "t5-small"

**Change Path** - Path for model

In [ ]:
# parameters of Seq2SeqTrainingArguments
batch_size = 4
num_training_epochs = 15
logging_steps = len(tokenized_dataset["train"])//4
model_name = "t5-small-finetuned-MCS-3"
model_dir = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/fine-tuned_models/t5-small-finetuned-MCS-3.h5"

args = Seq2SeqTrainingArguments(evaluation_strategy='steps',
                                learning_rate=5.6e-5,
                                per_device_train_batch_size=batch_size,
                                per_device_eval_batch_size=batch_size,
                                weight_decay=0.01,
                                eval_steps=2,
                                save_steps=2,
                                save_total_limit=3,
                                num_train_epochs=num_training_epochs,
                                predict_with_generate=True,
                                logging_steps=logging_steps,
                                output_dir="t5-small-finetuned-MCS-3",
                                load_best_model_at_end=True,
                                metric_for_best_model='eval_loss',
                                greater_is_better=False,
                                push_to_hub=False)


In [ ]:
import evaluate
rouge_score = evaluate.load('rouge')
def compute_metric(eval_pred):
  predictions, labels = eval_pred
  # Decode generated summaries into text (summaries generated from the review_body)
  # batch_decode -> Returns List[str]; The list of decoded sentences; Convert a list of lists of token ids into a list of strings by calling decode.
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True) # skip_special_tokens - Whether or not to remove special tokens in the decoding.
  # Replace -100 in the labels as we can't decode them
  labels = np.where(labels!=-100, labels, tokenizer.pad_token_id)
  # Decode reference summaries into text ('review_title')
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  # ROUGE expects a newline after each sentence
  # strip() method removes characters from both left and right based on the argument
  decoded_preds = ['\n'.join(sent_tokenize(pred.strip())) for pred in decoded_preds]
  decoded_labels = ['\n'.join(sent_tokenize(pred.strip())) for pred in decoded_labels]
  # Computing ROUGE score
  # Stemming is used to remove plurals and word suffixes such as (ing, ion, ment)
  result = rouge_score.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  result = {key: value*100 for key, value in result.items()}
  return {k: round(v,4) for k, v in result.items()}

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
datasets['train'].column_names, len(datasets['train'].column_names)

(['data', 'summary'], 2)

In [ ]:
tokenized_dataset = tokenized_dataset.remove_columns(datasets['train'].column_names)
tokenized_dataset['train'].column_names

['input_ids', 'attention_mask', 'labels']

In [ ]:
from transformers import EarlyStoppingCallback
# args = Seq2SeqTrainingArguments()
# model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')
# tokenizer = AutoTokenizer.from_pretrained('t5-small')
trainer = Seq2SeqTrainer(model, args, train_dataset=tokenized_dataset["train"], eval_dataset=tokenized_dataset["valid"],
                         data_collator=data_collator, tokenizer=tokenizer, compute_metrics=compute_metric,
                         callbacks=[EarlyStoppingCallback(early_stopping_patience=2)])

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 17
  Num Epochs = 15
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 75


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
2,No log,8.766941,5.337200,0.506300,5.188900,5.188900
4,7.606500,7.907529,6.464000,0.506300,5.885300,5.981000
6,7.606500,7.173917,6.139400,0.506300,5.520300,5.737600
8,5.893400,6.587154,6.139400,0.506300,5.520300,5.737600
10,5.893400,6.208655,5.450000,0.506300,4.956900,5.174200
12,5.365500,5.998305,5.871400,0.506300,5.332000,5.489100
14,5.365500,5.870716,5.871400,0.506300,5.332000,5.489100
16,4.817100,5.800640,6.326900,0.000000,5.332000,5.489100
18,4.817100,5.746162,6.326900,0.000000,5.332000,5.489100
20,4.768200,5.699948,6.326900,0.000000,5.332000,5.489100


***** Running Evaluation *****
  Num examples = 5
  Batch size = 4
Saving model checkpoint to t5-small-finetuned-MCS-3/checkpoint-2
Configuration saved in t5-small-finetuned-MCS-3/checkpoint-2/config.json
Model weights saved in t5-small-finetuned-MCS-3/checkpoint-2/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-MCS-3/checkpoint-2/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-MCS-3/checkpoint-2/special_tokens_map.json
Copy vocab file to t5-small-finetuned-MCS-3/checkpoint-2/spiece.model
***** Running Evaluation *****
  Num examples = 5
  Batch size = 4
Saving model checkpoint to t5-small-finetuned-MCS-3/checkpoint-4
Configuration saved in t5-small-finetuned-MCS-3/checkpoint-4/config.json
Model weights saved in t5-small-finetuned-MCS-3/checkpoint-4/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-MCS-3/checkpoint-4/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-MCS-3/checkpoint-4/special_tokens_map.jso

TrainOutput(global_step=75, training_loss=4.849761161804199, metrics={'train_runtime': 141.2415, 'train_samples_per_second': 1.805, 'train_steps_per_second': 0.531, 'total_flos': 34512159375360.0, 'train_loss': 4.849761161804199, 'epoch': 15.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 5
  Batch size = 4


{'eval_loss': 5.442809104919434,
 'eval_rouge1': 4.1502,
 'eval_rouge2': 0.0,
 'eval_rougeL': 4.4545,
 'eval_rougeLsum': 3.1846,
 'eval_runtime': 0.8866,
 'eval_samples_per_second': 5.64,
 'eval_steps_per_second': 2.256,
 'epoch': 15.0}

**Change Path** - Path of model saved in model training

In [ ]:
model_dir = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/fine-tuned_models/t5-small-finetuned-MCS-3.h5"
trainer.save_model(model_dir)

Saving model checkpoint to /content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/fine-tuned_models/t5-small-finetuned-MCS-3.h5
Configuration saved in /content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/fine-tuned_models/t5-small-finetuned-MCS-3.h5/config.json
Model weights saved in /content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/fine-tuned_models/t5-small-finetuned-MCS-3.h5/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/fine-tuned_models/t5-small-finetuned-MCS-3.h5/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/fine-tuned_models/t5-small-finetuned-MCS-3.h5/special_tokens_map.json
Copy vocab file to /content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/fine-tuned_models/t5-small-finetuned-MCS-3.h5/spiece.model


## BART Utilities

In [ ]:
# This run uses Pytorch Lightening to finetune the model
!pip install -q pytorch-lightning
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.4/722.4 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.8/728.8 kB 38.0 MB/s eta 0:00:00


In [ ]:
import glob
from nltk import tokenize
import nltk
import transformers
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import pandas as pd
import numpy as np
import torch.nn.functional as F
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint

#Source - https://colab.research.google.com/drive/1Cy27V-7qqYatqMA7fEqG2kgMySZXw9I4?usp=sharing&pli=1
class LitModel(pl.LightningModule):
  # Instantiate the model
  def __init__(self, learning_rate, tokenizer, model):
    super().__init__()
    self.tokenizer = tokenizer
    self.model = model
    self.learning_rate = learning_rate
    # self.freeze_encoder = freeze_encoder
    # self.freeze_embeds_ = freeze_embeds
#     self.hparams = argparse.Namespace()

    self.hparams.freeze_encoder = True
    self.hparams.freeze_embeds = True
    self.hparams.eval_beams = 4
    # self.hparams = hparams

    if self.hparams.freeze_encoder:
      freeze_params(self.model.get_encoder())

    if self.hparams.freeze_embeds:
      self.freeze_embeds()

  def freeze_embeds(self):
    ''' freeze the positional embedding parameters of the model; adapted from finetune.py '''
    freeze_params(self.model.model.shared)
    for d in [self.model.model.encoder, self.model.model.decoder]:
      freeze_params(d.embed_positions)
      freeze_params(d.embed_tokens)

  # Do a forward pass through the model
  def forward(self, input_ids, **kwargs):
    return self.model(input_ids, **kwargs)

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr = self.learning_rate)
    return optimizer

  def training_step(self, batch, batch_idx):
    # Load the data into variables
    src_ids, src_mask = batch[0], batch[1]
    tgt_ids = batch[2]
    # Shift the decoder tokens right (but NOT the tgt_ids)
    decoder_input_ids = shift_tokens_right(tgt_ids, self.tokenizer.pad_token_id)

    # Run the model and get the logits
    outputs = self(src_ids, attention_mask=src_mask, decoder_input_ids=decoder_input_ids, use_cache=False)
    lm_logits = outputs[0]
    # Create the loss function
    ce_loss_fct = torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
    # Calculate the loss on the un-shifted tokens
    loss = ce_loss_fct(lm_logits.view(-1, lm_logits.shape[-1]), tgt_ids.view(-1))

    return {'loss':loss}

  def validation_step(self, batch, batch_idx):

    src_ids, src_mask = batch[0], batch[1]
    tgt_ids = batch[2]

    decoder_input_ids = shift_tokens_right(tgt_ids, self.tokenizer.pad_token_id)

    # Run the model and get the logits
    outputs = self(src_ids, attention_mask=src_mask, decoder_input_ids=decoder_input_ids, use_cache=False)
    lm_logits = outputs[0]

    ce_loss_fct = torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
    val_loss = ce_loss_fct(lm_logits.view(-1, lm_logits.shape[-1]), tgt_ids.view(-1))

    return {'loss': val_loss}

  # Method that generates text using the BartForConditionalGeneration's generate() method
  def generate_text(self, text, eval_beams, early_stopping = True, max_len = 1024):
    ''' Function to generate text '''
    generated_ids = self.model.generate(
        text["input_ids"],
        attention_mask=text["attention_mask"],
        use_cache=True,
        decoder_start_token_id = self.tokenizer.pad_token_id,
        num_beams= eval_beams,
        max_length = max_len,
        early_stopping = early_stopping
    )
    return [self.tokenizer.decode(w, skip_special_tokens=True, clean_up_tokenization_spaces=True) for w in generated_ids]

def freeze_params(model):
  ''' Function that takes a model as input (or part of a model) and freezes the layers for faster training
      adapted from finetune.py '''
  for layer in model.parameters():
    layer.requires_grade = False

In [ ]:
# Create a dataloading module as per the PyTorch Lightning Docs
class SummaryDataModule(pl.LightningDataModule):
  def __init__(self, tokenizer, df, batch_size):
    super().__init__()
    self.tokenizer = tokenizer
    self.batch_size = batch_size
    self.data = df

  # Loads and splits the data into training, validation and test sets with a 60/20/20 split
  def prepare_data(self):
    self.train, self.validate, self.test = np.split(self.data.sample(frac=1), [int(.6*len(self.data)), int(.8*len(self.data))])

  # encode the sentences using the tokenizer
  def setup(self, stage):
    self.train = encode_sentences(self.tokenizer, self.train['source'], self.train['target'])
    self.validate = encode_sentences(self.tokenizer, self.validate['source'], self.validate['target'])
    self.test = encode_sentences(self.tokenizer, self.test['source'], self.test['target'])

  # Load the training, validation and test sets in Pytorch Dataset objects
  def train_dataloader(self):
    dataset = TensorDataset(self.train['input_ids'], self.train['attention_mask'], self.train['labels'])
    train_data = DataLoader(dataset, sampler = RandomSampler(dataset), batch_size = self.batch_size)
    return train_data

  def val_dataloader(self):
    dataset = TensorDataset(self.validate['input_ids'], self.validate['attention_mask'], self.validate['labels'])
    val_data = DataLoader(dataset, batch_size = self.batch_size)
    return val_data

  def test_dataloader(self):
    dataset = TensorDataset(self.test['input_ids'], self.test['attention_mask'], self.test['labels'])
    test_data = DataLoader(dataset, batch_size = self.batch_size)
    return test_data

In [ ]:
def shift_tokens_right(input_ids, pad_token_id):
  """ Shift input ids one token to the right, and wrap the last non pad token (usually <eos>).
      This is taken directly from modeling_bart.py
  """
  prev_output_tokens = input_ids.clone()
  index_of_eos = (input_ids.ne(pad_token_id).sum(dim=1) - 1).unsqueeze(-1)
  prev_output_tokens[:, 0] = input_ids.gather(1, index_of_eos).squeeze()
  prev_output_tokens[:, 1:] = input_ids[:, :-1]
  return prev_output_tokens

def encode_sentences(tokenizer, source_sentences, target_sentences, max_length=1024, min_length = 512, pad_to_max_length=True, return_tensors="pt"):
  ''' Function that tokenizes a sentence
      Args: tokenizer - the BART tokenizer; source and target sentences are the source and target sentences
      Returns: Dictionary with keys: input_ids, attention_mask, target_ids
  '''

  input_ids = []
  attention_masks = []
  target_ids = []
  tokenized_sentences = {}

  for sentence in source_sentences:
    encoded_dict = tokenizer(
          sentence,
          max_length=max_length,
          padding="max_length" if pad_to_max_length else None,
          truncation=True,
          return_tensors=return_tensors,
          add_prefix_space = True
      )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

  input_ids = torch.cat(input_ids, dim = 0)
  attention_masks = torch.cat(attention_masks, dim = 0)

  for sentence in target_sentences:
    encoded_dict = tokenizer(
          sentence,
          max_length=min_length,
          padding="max_length" if pad_to_max_length else None,
          truncation=True,
          return_tensors=return_tensors,
          add_prefix_space = True
      )
    # Shift the target ids to the right
    # shifted_target_ids = shift_tokens_right(encoded_dict['input_ids'], tokenizer.pad_token_id)
    target_ids.append(encoded_dict['input_ids'])

  target_ids = torch.cat(target_ids, dim = 0)


  batch = {
      "input_ids": input_ids,
      "attention_mask": attention_masks,
      "labels": target_ids,
  }

  return batch

## Fine-Tuning BART

In [ ]:
filepath = "/content/mcs.xlsx"
df = pd.read_excel(filepath,usecols='B,C')
#df = pd.read_excel(filename,index_col=0)
df.rename(columns = {'data':'source', 'summary':'target'}, inplace = True)

In [ ]:
df.head()

,source,target
0,Appeal No. LXVI of 1949. Appeal from the High ...,The charge created in respect of municipal pro...
1,"Dudhuria vs Commissioner of Income tax, Calcut...",The charge in respect of urban immoveable prop...
2,"Kania J., as he then was, said as follows : ""I...","The expression ""capital charge"" in s.9(1) (iv)..."


In [ ]:
# Loading Model and tokenizer
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, BartConfig

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large', add_prefix_space=True)

bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")

In [ ]:
# Add special tokens if required

new_tokens = ['<F>', '<RLC>', '<A>', '<S>', '<P>', '<R>', '<RPC>']

special_tokens_dict = {'additional_special_tokens': new_tokens}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
bart_model.resize_token_embeddings(len(tokenizer))

Embedding(50272, 1024)

In [ ]:
summary_data = SummaryDataModule(tokenizer, df, batch_size = 1)
model = LitModel(learning_rate = 2e-5, tokenizer = tokenizer, model = bart_model)

In [ ]:
# put in chatgpt for code implementation - "finetuning BART model with LitModel multi GPU implementation with tensorflow"
# for early stopping - "finetuning BART model with LitModel and adding a early stopping criterea with lightning module"

In [ ]:
# import pytorch_lightning as pl

trainer = pl.Trainer(accelerator="auto", # "accelarator='auto'" keyword which automatically uses all available GPUs
                     max_epochs = 3,
                     min_epochs = 2,
                     precision = 16)

INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, summary_data)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                         | Params
-------------------------------------------------------
0 | model | BartForConditionalGeneration | 406 M 
-------------------------------------------------------
406 M     Trainable params
0         Non-trainable params
406 M     Total params
1,625.194 Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


In [ ]:
#Save the model
trainer.save_checkpoint("output.ckpt")

## Evaluation (Extractive) of Pre-Trained BART

In [ ]:
!pip install evaluate
!pip install bert_score

In [ ]:
import sys
sys.path.insert(0, '../')
import glob
import os
import json
import pandas as pd
import nltk
nltk.download('punkt')
from rouge_score import rouge_scorer
from rouge import Rouge
import math
import numpy as np
from tqdm import tqdm
from evaluate import load

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
ext_analysis = pd.read_csv('Extractive Test Dataset Analysis.csv')

In [ ]:
names, summ_word_count = list(ext_analysis['File']), list(ext_analysis['L = Average Summary Word Count'])

get_req_len = {}
for i in range(len(names)):
  get_req_len[names[i]] = summ_word_count[i]

In [ ]:
get_req_len

{'2012_S_270.txt': 1362,
 '2014_J_33.txt': 1007,
 '2014_R_41.txt': 1078,
 '2015_J_10.txt': 1126,
 '2015_S_368.txt': 1400}

In [ ]:
# Loading Model and tokenizer
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, BartConfig

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large', add_prefix_space=True)


model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")

In [ ]:
bart_model = LitModel(learning_rate = 2e-5, tokenizer = tokenizer, model = model)

# bart_model = LitModel.load_from_checkpoint("/home/pahelibhattacharya/HULK/Abhay/models/BART_large_IN_MCS.ckpt",
#                                       learning_rate = 2e-5, tokenizer = tokenizer, model = model).to("cuda")

In [ ]:
def nest_sentences(document,chunk_length):
    '''
    function to chunk a document
    input:  document           - Input document
            chunk_length        - chunk length
    output: list of chunks. Each chunk is a string.
    '''
    nested = []  # to store list of chunks of a doc
    sent = []    # to store sentences per chunk in loop
    length = 0
    for sentence in nltk.sent_tokenize(document):
        length += len(sentence.split(" "))       # adding no of words per sentence
        if length < chunk_length:
            sent.append(sentence)
        else:
            nested.append(" ".join(sent))        # when word limit exceed chunk length (512 for t5) add the list of sentences (a chunk) after joining to the nested list
            sent = []                            # create a new list to store next batch fof sentences for subsequent chunk
            sent.append(sentence)
            length = 0
            length += len(sentence.split(" "))   # added to include length i=of sentence in else loop. Initially not added
    if len(sent)>0:
        nested.append(" ".join(sent))            # including the final list of sentences to nested
    return nested                                # return the list of chunks of that doc

In [ ]:
def generate_summary_gpu(nested_sentences,p=0.2):
  '''
    Function to generate summaries from the list containing chunks of the document
    input:  nested_sentences - chunks
            p - Number of words in summaries per word in the document
    output: document summary
    '''
  summaries = []
  for nested in nested_sentences:         # looping through each of the chunks
    l = int(p * len(nested.split(" ")))   # multiplying p with length of chunk to distribute summary length to that particular chunk
    input_tokenized = tokenizer.encode(nested, truncation=True, return_tensors='pt')
    summary_ids = bart_model.model.generate(input_tokenized,
                                      length_penalty=0.01,
                                      min_length=l-5,
                                      max_length=l+5)
    output = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    summaries.append(output)
  summaries = [sentence for sublist in summaries for sentence in sublist]
  print(summaries)
  return summaries

### Testing A1

In [ ]:
def get_root_path():
    '''
    function to get root path of dataset
    change the path variable to the path of the dataset
    '''
    path = "/content/drive/MyDrive/ExtractiveDataset/test-data"  # use all 50 docs for testing if using abstractive dataset for training
    return path

def get_summary_data():
    '''
    function to get names, documents, and summaries
    change the path variable to the path of the dataset
    '''

    path = get_root_path() + '/judgement'
    all_files = glob.glob(path + "/*.txt")
    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f:
            p = filename.rfind("/")
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)
    path = get_root_path() + '/summary/A1'
    all_files = glob.glob(path + "/*.txt")
    data_summary = []
    for filename in all_files:
        with open(filename, 'r') as f:
            a = f.read()
            l = len(a)
            data_summary.append(a)

    return names, data_source, data_summary

def split_to_sentences(para):
    sents = nltk.sent_tokenize(para)   # returns list of sentences from para
    return sents

In [ ]:
names, data_source, data_summary = get_summary_data()
print(len(names))
print(len(data_source))
print(len(data_summary))


5
5
5


In [ ]:
# main loop to generate and save summaries of each document in the test dataset

result_rouge_list = []
result_bertscore_list = []

for i in tqdm(range(2)):  # range(len(data_source))
    name = names[i]
    doc = data_source[i]
    input_len = len(doc.split(" "))
    req_len = get_req_len[name]
    print(str(i) + ": " + name +  " - " + str(input_len) + " : " + str(req_len), end = " ,")

    nested = nest_sentences(doc,512)  # returns a list of chunks with word count < 512
    p = float(req_len/input_len)  # p is the ratio of summary length to doc length
    print('Value of p for doc {} is {}'.format(i,p))

    extr_summ = generate_summary_gpu(nested,p)  # returns list of chunks summaries
    print('--------------------------')

    extr_summ = " ".join(extr_summ)
    print(extr_summ)

    if len(extr_summ.split(" ")) > req_len:
        extr_summ = extr_summ.split(" ")
        extr_summ = extr_summ[:req_len]
        extr_summ = " ".join(extr_summ)

    print(len(data_summary[i].split(' ')),len(extr_summ.split(' ')))

    rouge = Rouge()
    result_rouge_list.append(rouge.get_scores(extr_summ, data_summary[i], avg=True))
    bertscore = load("bertscore")
    result_bertscore_list.append(bertscore.compute(predictions=[extr_summ], references=[data_summary[i]], model_type="distilbert-base-uncased"))
    print('---------------------------------------------------------------------------------------------------------------')

  0%|          | 0/2 [00:00<?, ?it/s]

0: 2012_S_270.txt - 4248 : 1362 ,Value of p for doc 0 is 0.3206214689265537


  0%|          | 0/2 [00:01<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
hypothesis_1 = "King Norodom Sihanouk has declined requests to chair a summit of Cambodia 's top political leaders , saying the meeting would not bring any progress in deadlocked negotiations to form a government .\nGovernment and opposition parties have asked King Norodom Sihanouk to host a summit meeting after a series of post-election negotiations between the two opposition groups and Hun Sen 's party to form a new government failed .\nHun Sen 's ruling party narrowly won a majority in elections in July , but the opposition _ claiming widespread intimidation and fraud _ has denied Hun Sen the two-thirds vote in parliament required to approve the next government .\n"
references_1 = ["Prospects were dim for resolution of the political crisis in Cambodia in October 1998.\nPrime Minister Hun Sen insisted that talks take place in Cambodia while opposition leaders Ranariddh and Sam Rainsy, fearing arrest at home, wanted them abroad.\nKing Sihanouk declined to chair talks in either place.\nA U.S. House resolution criticized Hun Sen's regime while the opposition tried to cut off his access to loans.\nBut in November the King announced a coalition government with Hun Sen heading the executive and Ranariddh leading the parliament.\nLeft out, Sam Rainsy sought the King's assurance of Hun Sen's promise of safety and freedom for all politicians.",
                "Cambodian prime minister Hun Sen rejects demands of 2 opposition parties for talks in Beijing after failing to win a 2/3 majority in recent elections.\nSihanouk refuses to host talks in Beijing.\nOpposition parties ask the Asian Development Bank to stop loans to Hun Sen's government.\nCCP defends Hun Sen to the US Senate.\nFUNCINPEC refuses to share the presidency.\nHun Sen and Ranariddh eventually form a coalition at summit convened by Sihanouk.\nHun Sen remains prime minister, Ranariddh is president of the national assembly, and a new senate will be formed.\nOpposition leader Rainsy left out.\nHe seeks strong assurance of safety should he return to Cambodia.\n",
                ]

hypothesis_2 = "China 's government said Thursday that two prominent dissidents arrested this week are suspected of endangering national security _ the clearest sign yet Chinese leaders plan to quash a would-be opposition party .\nOne leader of a suppressed new political party will be tried on Dec. 17 on a charge of colluding with foreign enemies of China '' to incite the subversion of state power , '' according to court documents given to his wife on Monday .\nWith attorneys locked up , harassed or plain scared , two prominent dissidents will defend themselves against charges of subversion Thursday in China 's highest-profile dissident trials in two years .\n"
references_2 = "Hurricane Mitch, category 5 hurricane, brought widespread death and destruction to Central American.\nEspecially hard hit was Honduras where an estimated 6,076 people lost their lives.\nThe hurricane, which lingered off the coast of Honduras for 3 days before moving off, flooded large areas, destroying crops and property.\nThe U.S. and European Union were joined by Pope John Paul II in a call for money and workers to help the stricken area.\nPresident Clinton sent Tipper Gore, wife of Vice President Gore to the area to deliver much needed supplies to the area, demonstrating U.S. commitment to the recovery of the region.\n"

all_hypothesis = [hypothesis_1, hypothesis_2]
all_references = [references_1, references_2]

scores = evaluator.get_scores(all_hypothesis, all_references)

In [ ]:
agg_rouge_score_1 = {'rouge-1':{'r':0,'p':0,'f':0},'rouge-2':{'r':0,'p':0,'f':0},
                   'rouge-l':{'r':0,'p':0,'f':0}}

for i in range(len(result_rouge_list)):
  for rouge_type in ['rouge-1','rouge-2','rouge-l']:
    for j in ['r','p','f']:
      agg_rouge_score_1[rouge_type][j]+=(result_rouge_list[i][rouge_type][j]/len(result_rouge_list))

print(agg_rouge_score_1)

{'rouge-1': {'r': 0.34919517102615694, 'p': 0.4248799292214358, 'f': 0.38269036953218116}, 'rouge-2': {'r': 0.17655071776181952, 'p': 0.19880821541420524, 'f': 0.1858949298444496}, 'rouge-l': {'r': 0.3137683242311009, 'p': 0.3802641557128412, 'f': 0.3432646344228572}}


In [ ]:
agg_bertscore_1 = {'precision':0, 'recall':0, 'f1':0}

for i in range(len(result_bertscore_list)):
  for metric_type in ['precision','recall','f1']:
    agg_bertscore_1[metric_type]+=((result_bertscore_list[i][metric_type][0])/len(result_bertscore_list))

print(agg_bertscore_1)

{'precision': 0.7688697874546051, 'recall': 0.7728851437568665, 'f1': 0.7708407342433929}


### Testing A2

In [ ]:
def get_root_path():
    '''
    function to get root path of dataset
    change the path variable to the path of the dataset
    '''
    path = "/content/drive/MyDrive/ExtractiveDataset/test-data"  # use all 50 docs for testing if using abstractive dataset for training
    return path

def get_summary_data():
    '''
    function to get names, documents, and summaries
    change the path variable to the path of the dataset
    '''

    path = get_root_path() + '/judgement'
    all_files = glob.glob(path + "/*.txt")
    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f:
            p = filename.rfind("/")
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)
    path = get_root_path() + '/summary/A2'
    all_files = glob.glob(path + "/*.txt")
    data_summary = []
    for filename in all_files:
        with open(filename, 'r') as f:
            a = f.read()
            l = len(a)
            data_summary.append(a)

    return names, data_source, data_summary

def split_to_sentences(para):
    sents = nltk.sent_tokenize(para)   # returns list of sentences from para
    return sents

In [ ]:
names, data_source, data_summary = get_summary_data()
print(len(names))
print(len(data_source))
print(len(data_summary))


5
5
5


In [ ]:
# main loop to generate and save summaries of each document in the test dataset

result_rouge_list = []
result_bertscore_list = []

for i in tqdm(range(len(data_source))):
    name = names[i]
    doc = data_source[i]
    input_len = len(doc.split(" "))
    req_len = get_req_len[name]
    print(str(i) + ": " + name +  " - " + str(input_len) + " : " + str(req_len), end = " ,")

    nested = nest_sentences(doc,512)  # returns a list of chunks with word count < 512
    p = float(req_len/input_len)  # p is the ratio of summary length to doc length
    print('Value of p for doc {} is {}'.format(i,p))

    extr_summ = generate_summary_gpu(nested,p)  # returns list of chunks summaries

    extr_summ = " ".join(extr_summ)

    if len(extr_summ.split(" ")) > req_len:
        extr_summ = extr_summ.split(" ")
        extr_summ = extr_summ[:req_len]
        extr_summ = " ".join(extr_summ)

    print(len(data_summary[i].split(' ')),len(extr_summ.split(' ')))

    rouge = Rouge()
    result_rouge_list.append(rouge.get_scores(extr_summ, data_summary[i], avg=True))
    bertscore = load("bertscore")
    result_bertscore_list.append(bertscore.compute(predictions=[extr_summ], references=[data_summary[i]], model_type="distilbert-base-uncased"))
    print('---------------------------------------------------------------------------------------------------------------')

  0%|          | 0/5 [00:00<?, ?it/s]

0: 2012_S_270.txt - 4248 : 1362 ,Value of p for doc 0 is 0.3206214689265537


In [ ]:
agg_rouge_score_2 = {'rouge-1':{'r':0,'p':0,'f':0},'rouge-2':{'r':0,'p':0,'f':0},
                   'rouge-l':{'r':0,'p':0,'f':0}}

for i in range(len(result_rouge_list)):
  for rouge_type in ['rouge-1','rouge-2','rouge-l']:
    for j in ['r','p','f']:
      agg_rouge_score_2[rouge_type][j]+=(result_rouge_list[i][rouge_type][j]/len(result_rouge_list))

print(agg_rouge_score_2)

{'rouge-1': {'r': 0.21187794059777082, 'p': 0.49926828337230633, 'f': 0.2935809851176972}, 'rouge-2': {'r': 0.07604486106863347, 'p': 0.20917546307120924, 'f': 0.1102924148470025}, 'rouge-l': {'r': 0.20117182810722975, 'p': 0.47589578811019456, 'f': 0.2791278810436789}}


In [ ]:
agg_bertscore_2 = {'precision':0, 'recall':0, 'f1':0}

for i in range(len(result_bertscore_list)):
  for metric_type in ['precision','recall','f1']:
    agg_bertscore_2[metric_type]+=((result_bertscore_list[i][metric_type][0])/len(result_bertscore_list))

print(agg_bertscore_2)

{'precision': 0.7443345665931702, 'recall': 0.751188361644745, 'f1': 0.7473442792892456}


###Average ROUGE score and BERTScores

In [ ]:
agg_rouge_score = {'rouge-1':{'r':0,'p':0,'f':0},'rouge-2':{'r':0,'p':0,'f':0},
                   'rouge-l':{'r':0,'p':0,'f':0}}

for rouge_type in ['rouge-1','rouge-2','rouge-l']:
  for j in ['r','p','f']:
    agg_rouge_score[rouge_type][j]=(agg_rouge_score_1[rouge_type][j]+agg_rouge_score_2[rouge_type][j])*0.5

print(agg_rouge_score)

In [ ]:
print('ROUGE Scores of A1 Golden summary:')
print(agg_rouge_score_1)
print()
print('ROUGE Scores of A2 Golden summary:')
print(agg_rouge_score_2)
print()
print('ROUGE Average ROUGE scores of both summaries:')
print(agg_rouge_score)

In [ ]:
agg_bertscore = {'precision':0, 'recall':0, 'f1':0}

for metric_type in ['precision','recall','f1']:
  agg_bertscore[metric_type]+=(agg_bertscore_1[metric_type]+agg_bertscore_2[metric_type])*0.5

print(agg_bertscore)

In [ ]:
print('BERTScores of A1 Golden summary:')
print(agg_bertscore_1)
print()
print('BERTScores of A2 Golden summary:')
print(agg_bertscore_2)
print()
print('Average BERTScores scores of both summaries:')
print(agg_bertscore)

## Evaluation (Extractive) of Fine-Tuned BART

In [ ]:
!pip install evaluate
!pip install bert_score

In [ ]:
import sys
sys.path.insert(0, '../')
import glob
import os
import json
import pandas as pd
import nltk
nltk.download('punkt')
from rouge_score import rouge_scorer
from rouge import Rouge
import math
import numpy as np
from tqdm import tqdm
from evaluate import load

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
ext_analysis = pd.read_csv('Extractive Test Dataset Analysis.csv')

In [ ]:
names, summ_word_count = list(ext_analysis['File']), list(ext_analysis['L = Average Summary Word Count'])

get_req_len = {}
for i in range(len(names)):
  get_req_len[names[i]] = summ_word_count[i]

In [ ]:
get_req_len

{'2012_S_270.txt': 1362,
 '2014_J_33.txt': 1007,
 '2014_R_41.txt': 1078,
 '2015_J_10.txt': 1126,
 '2015_S_368.txt': 1400}

In [ ]:
# Loading Model and tokenizer
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, BartConfig

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large', add_prefix_space=True)


model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")

In [ ]:
# Add special tokens if required

new_tokens = ['<F>', '<RLC>', '<A>', '<S>', '<P>', '<R>', '<RPC>']

special_tokens_dict = {'additional_special_tokens': new_tokens}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
bart_model.resize_token_embeddings(len(tokenizer))

Embedding(50272, 1024)

In [ ]:
#bart_model = LitModel(learning_rate = 2e-5, tokenizer = tokenizer, model = model)

bart_model = LitModel.load_from_checkpoint("output.ckpt", learning_rate = 2e-5, tokenizer = tokenizer, model = bart_model)

In [ ]:
def nest_sentences(document,chunk_length):
    '''
    function to chunk a document
    input:  document           - Input document
            chunk_length        - chunk length
    output: list of chunks. Each chunk is a string.
    '''
    nested = []  # to store list of chunks of a doc
    sent = []    # to store sentences per chunk in loop
    length = 0
    for sentence in nltk.sent_tokenize(document):
        length += len(sentence.split(" "))       # adding no of words per sentence
        if length < chunk_length:
            sent.append(sentence)
        else:
            nested.append(" ".join(sent))        # when word limit exceed chunk length (512 for t5) add the list of sentences (a chunk) after joining to the nested list
            sent = []                            # create a new list to store next batch fof sentences for subsequent chunk
            sent.append(sentence)
            length = 0
            length += len(sentence.split(" "))   # added to include length i=of sentence in else loop. Initially not added
    if len(sent)>0:
        nested.append(" ".join(sent))            # including the final list of sentences to nested
    return nested                                # return the list of chunks of that doc

In [ ]:
def generate_summary_gpu(nested_sentences,p=0.2):
  '''
    Function to generate summaries from the list containing chunks of the document
    input:  nested_sentences - chunks
            p - Number of words in summaries per word in the document
    output: document summary
    '''
  summaries = []
  for nested in nested_sentences:         # looping through each of the chunks
    l = int(p * len(nested.split(" ")))   # multiplying p with length of chunk to distribute summary length to that particular chunk
    input_tokenized = tokenizer.encode(nested, truncation=True, return_tensors='pt')
    summary_ids = bart_model.model.generate(input_tokenized,
                                      length_penalty=0.01,
                                      min_length=l-5,
                                      max_length=l+5)
    output = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    summaries.append(output)
  summaries = [sentence for sublist in summaries for sentence in sublist]
  return summaries

### Testing A1

In [ ]:
def get_root_path():
    '''
    function to get root path of dataset
    change the path variable to the path of the dataset
    '''
    path = "/content/drive/MyDrive/ExtractiveDataset/test-data"  # use all 50 docs for testing if using abstractive dataset for training
    return path

def get_summary_data():
    '''
    function to get names, documents, and summaries
    change the path variable to the path of the dataset
    '''

    path = get_root_path() + '/judgement'
    all_files = glob.glob(path + "/*.txt")
    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f:
            p = filename.rfind("/")
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)
    path = get_root_path() + '/summary/A1'
    all_files = glob.glob(path + "/*.txt")
    data_summary = []
    for filename in all_files:
        with open(filename, 'r') as f:
            a = f.read()
            l = len(a)
            data_summary.append(a)

    return names, data_source, data_summary

def split_to_sentences(para):
    sents = nltk.sent_tokenize(para)   # returns list of sentences from para
    return sents

In [ ]:
names, data_source, data_summary = get_summary_data()
print(len(names))
print(len(data_source))
print(len(data_summary))


In [ ]:
# main loop to generate and save summaries of each document in the test dataset

result_rouge_list = []
result_bertscore_list = []

for i in tqdm(range(2)):  # range(len(data_source))
    name = names[i]
    doc = data_source[i]
    input_len = len(doc.split(" "))
    req_len = get_req_len[name]
    print(str(i) + ": " + name +  " - " + str(input_len) + " : " + str(req_len), end = " ,")

    nested = nest_sentences(doc,512)  # returns a list of chunks with word count < 512
    p = float(req_len/input_len)  # p is the ratio of summary length to doc length
    print('Value of p for doc {} is {}'.format(i,p))

    extr_summ = generate_summary_gpu(nested,p)  # returns list of chunks summaries

    extr_summ = " ".join(extr_summ)

    if len(extr_summ.split(" ")) > req_len:
        extr_summ = extr_summ.split(" ")
        extr_summ = extr_summ[:req_len]
        extr_summ = " ".join(extr_summ)

    print(len(data_summary[i].split(' ')),len(extr_summ.split(' ')))

    rouge = Rouge()
    result_rouge_list.append(rouge.get_scores(extr_summ, data_summary[i], avg=True))
    bertscore = load("bertscore")
    result_bertscore_list.append(bertscore.compute(predictions=[extr_summ], references=[data_summary[i]], model_type="distilbert-base-uncased"))
    print('---------------------------------------------------------------------------------------------------------------')

  0%|          | 0/2 [00:00<?, ?it/s]

0: 2012_S_270.txt - 4248 : 1362 ,Value of p for doc 0 is 0.3206214689265537


  0%|          | 0/2 [00:14<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
agg_rouge_score_1 = {'rouge-1':{'r':0,'p':0,'f':0},'rouge-2':{'r':0,'p':0,'f':0},
                   'rouge-l':{'r':0,'p':0,'f':0}}

for i in range(len(result_rouge_list)):
  for rouge_type in ['rouge-1','rouge-2','rouge-l']:
    for j in ['r','p','f']:
      agg_rouge_score_1[rouge_type][j]+=(result_rouge_list[i][rouge_type][j]/len(result_rouge_list))

print(agg_rouge_score_1)

In [ ]:
agg_bertscore_1 = {'precision':0, 'recall':0, 'f1':0}

for i in range(len(result_bertscore_list)):
  for metric_type in ['precision','recall','f1']:
    agg_bertscore_1[metric_type]+=((result_bertscore_list[i][metric_type][0])/len(result_bertscore_list))

print(agg_bertscore_1)

### Testing A2

In [ ]:
def get_root_path():
    '''
    function to get root path of dataset
    change the path variable to the path of the dataset
    '''
    path = "/content/drive/MyDrive/ExtractiveDataset/test-data"  # use all 50 docs for testing if using abstractive dataset for training
    return path

def get_summary_data():
    '''
    function to get names, documents, and summaries
    change the path variable to the path of the dataset
    '''

    path = get_root_path() + '/judgement'
    all_files = glob.glob(path + "/*.txt")
    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f:
            p = filename.rfind("/")
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)
    path = get_root_path() + '/summary/A2'
    all_files = glob.glob(path + "/*.txt")
    data_summary = []
    for filename in all_files:
        with open(filename, 'r') as f:
            a = f.read()
            l = len(a)
            data_summary.append(a)

    return names, data_source, data_summary

def split_to_sentences(para):
    sents = nltk.sent_tokenize(para)   # returns list of sentences from para
    return sents

In [ ]:
names, data_source, data_summary = get_summary_data()
print(len(names))
print(len(data_source))
print(len(data_summary))


5
5
5


In [ ]:
# main loop to generate and save summaries of each document in the test dataset

result_rouge_list = []
result_bertscore_list = []

for i in tqdm(range(len(data_source))):
    name = names[i]
    doc = data_source[i]
    input_len = len(doc.split(" "))
    req_len = get_req_len[name]
    print(str(i) + ": " + name +  " - " + str(input_len) + " : " + str(req_len), end = " ,")

    nested = nest_sentences(doc,512)  # returns a list of chunks with word count < 512
    p = float(req_len/input_len)  # p is the ratio of summary length to doc length
    print('Value of p for doc {} is {}'.format(i,p))

    extr_summ = generate_summary_gpu(nested,p)  # returns list of chunks summaries

    extr_summ = " ".join(extr_summ)

    if len(extr_summ.split(" ")) > req_len:
        extr_summ = extr_summ.split(" ")
        extr_summ = extr_summ[:req_len]
        extr_summ = " ".join(extr_summ)

    print(len(data_summary[i].split(' ')),len(extr_summ.split(' ')))

    rouge = Rouge()
    result_rouge_list.append(rouge.get_scores(extr_summ, data_summary[i], avg=True))
    bertscore = load("bertscore")
    result_bertscore_list.append(bertscore.compute(predictions=[extr_summ], references=[data_summary[i]], model_type="distilbert-base-uncased"))
    print('---------------------------------------------------------------------------------------------------------------')

  0%|          | 0/5 [00:00<?, ?it/s]

0: 2012_S_270.txt - 4248 : 1362 ,Value of p for doc 0 is 0.3206214689265537


In [ ]:
agg_rouge_score_2 = {'rouge-1':{'r':0,'p':0,'f':0},'rouge-2':{'r':0,'p':0,'f':0},
                   'rouge-l':{'r':0,'p':0,'f':0}}

for i in range(len(result_rouge_list)):
  for rouge_type in ['rouge-1','rouge-2','rouge-l']:
    for j in ['r','p','f']:
      agg_rouge_score_2[rouge_type][j]+=(result_rouge_list[i][rouge_type][j]/len(result_rouge_list))

print(agg_rouge_score_2)

{'rouge-1': {'r': 0.21187794059777082, 'p': 0.49926828337230633, 'f': 0.2935809851176972}, 'rouge-2': {'r': 0.07604486106863347, 'p': 0.20917546307120924, 'f': 0.1102924148470025}, 'rouge-l': {'r': 0.20117182810722975, 'p': 0.47589578811019456, 'f': 0.2791278810436789}}


In [ ]:
agg_bertscore_2 = {'precision':0, 'recall':0, 'f1':0}

for i in range(len(result_bertscore_list)):
  for metric_type in ['precision','recall','f1']:
    agg_bertscore_2[metric_type]+=((result_bertscore_list[i][metric_type][0])/len(result_bertscore_list))

print(agg_bertscore_2)

{'precision': 0.7443345665931702, 'recall': 0.751188361644745, 'f1': 0.7473442792892456}


###Average ROUGE score and BERTScores

In [ ]:
agg_rouge_score = {'rouge-1':{'r':0,'p':0,'f':0},'rouge-2':{'r':0,'p':0,'f':0},
                   'rouge-l':{'r':0,'p':0,'f':0}}

for rouge_type in ['rouge-1','rouge-2','rouge-l']:
  for j in ['r','p','f']:
    agg_rouge_score[rouge_type][j]=(agg_rouge_score_1[rouge_type][j]+agg_rouge_score_2[rouge_type][j])*0.5

print(agg_rouge_score)

In [ ]:
print('ROUGE Scores of A1 Golden summary:')
print(agg_rouge_score_1)
print()
print('ROUGE Scores of A2 Golden summary:')
print(agg_rouge_score_2)
print()
print('ROUGE Average ROUGE scores of both summaries:')
print(agg_rouge_score)

In [ ]:
agg_bertscore = {'precision':0, 'recall':0, 'f1':0}

for metric_type in ['precision','recall','f1']:
  agg_bertscore[metric_type]+=(agg_bertscore_1[metric_type]+agg_bertscore_2[metric_type])*0.5

print(agg_bertscore)

In [ ]:
print('BERTScores of A1 Golden summary:')
print(agg_bertscore_1)
print()
print('BERTScores of A2 Golden summary:')
print(agg_bertscore_2)
print()
print('Average BERTScores scores of both summaries:')
print(agg_bertscore)

## Pegasus-X Utilities

In [ ]:
# This run uses Pytorch Lightening to finetune the model
!pip install -q pytorch-lightning
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.4/722.4 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 729.2/729.2 kB 63.8 MB/s eta 0:00:00


In [ ]:
import glob
from nltk import tokenize
import nltk
import transformers
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import pandas as pd
import numpy as np
import torch.nn.functional as F
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint

#Source - https://colab.research.google.com/drive/1Cy27V-7qqYatqMA7fEqG2kgMySZXw9I4?usp=sharing&pli=1
class LitModel(pl.LightningModule):
  # Instantiate the model
  def __init__(self, learning_rate, tokenizer, model):
    super().__init__()
    self.tokenizer = tokenizer
    self.model = model
    self.learning_rate = learning_rate
    # self.freeze_encoder = freeze_encoder
    # self.freeze_embeds_ = freeze_embeds
#     self.hparams = argparse.Namespace()

    self.hparams.freeze_encoder = True
    self.hparams.freeze_embeds = True
    self.hparams.eval_beams = 4
    # self.hparams = hparams

    if self.hparams.freeze_encoder:
      freeze_params(self.model.get_encoder())

    if self.hparams.freeze_embeds:
      self.freeze_embeds()

  def freeze_embeds(self):
    ''' freeze the positional embedding parameters of the model; adapted from finetune.py '''
    freeze_params(self.model.model.shared)
    for d in [self.model.model.encoder, self.model.model.decoder]:
      freeze_params(d.embed_positions)
      freeze_params(d.embed_tokens)

  # Do a forward pass through the model
  def forward(self, input_ids, **kwargs):
    return self.model(input_ids, **kwargs)

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr = self.learning_rate)
    return optimizer

  def training_step(self, batch, batch_idx):
    # Load the data into variables
    src_ids, src_mask = batch[0], batch[1]
    tgt_ids = batch[2]
    # Shift the decoder tokens right (but NOT the tgt_ids)
    decoder_input_ids = shift_tokens_right(tgt_ids, self.tokenizer.pad_token_id)

    # Run the model and get the logits
    outputs = self(src_ids, attention_mask=src_mask, decoder_input_ids=decoder_input_ids, use_cache=False)
    lm_logits = outputs[0]
    # Create the loss function
    ce_loss_fct = torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
    # Calculate the loss on the un-shifted tokens
    loss = ce_loss_fct(lm_logits.view(-1, lm_logits.shape[-1]), tgt_ids.view(-1))

    return {'loss':loss}

  def validation_step(self, batch, batch_idx):

    src_ids, src_mask = batch[0], batch[1]
    tgt_ids = batch[2]

    decoder_input_ids = shift_tokens_right(tgt_ids, self.tokenizer.pad_token_id)

    # Run the model and get the logits
    outputs = self(src_ids, attention_mask=src_mask, decoder_input_ids=decoder_input_ids, use_cache=False)
    lm_logits = outputs[0]

    ce_loss_fct = torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
    val_loss = ce_loss_fct(lm_logits.view(-1, lm_logits.shape[-1]), tgt_ids.view(-1))

    return {'loss': val_loss}

  # Method that generates text using the BartForConditionalGeneration's generate() method
  def generate_text(self, text, eval_beams, early_stopping = True, max_len = 1024):
    ''' Function to generate text '''
    generated_ids = self.model.generate(
        text["input_ids"],
        attention_mask=text["attention_mask"],
        use_cache=True,
        decoder_start_token_id = self.tokenizer.pad_token_id,
        num_beams= eval_beams,
        max_length = max_len,
        early_stopping = early_stopping
    )
    return [self.tokenizer.decode(w, skip_special_tokens=True, clean_up_tokenization_spaces=True) for w in generated_ids]

def freeze_params(model):
  ''' Function that takes a model as input (or part of a model) and freezes the layers for faster training
      adapted from finetune.py '''
  for layer in model.parameters():
    layer.requires_grade = False

In [ ]:
# Create a dataloading module as per the PyTorch Lightning Docs
class SummaryDataModule(pl.LightningDataModule):
  def __init__(self, tokenizer, df, batch_size):
    super().__init__()
    self.tokenizer = tokenizer
    self.batch_size = batch_size
    self.data = df

  # Loads and splits the data into training, validation and test sets with a 60/20/20 split
  def prepare_data(self):
    self.train, self.validate, self.test = np.split(self.data.sample(frac=1), [int(.6*len(self.data)), int(.8*len(self.data))])

  # encode the sentences using the tokenizer
  def setup(self, stage):
    self.train = encode_sentences(self.tokenizer, self.train['source'], self.train['target'])
    self.validate = encode_sentences(self.tokenizer, self.validate['source'], self.validate['target'])
    self.test = encode_sentences(self.tokenizer, self.test['source'], self.test['target'])

  # Load the training, validation and test sets in Pytorch Dataset objects
  def train_dataloader(self):
    dataset = TensorDataset(self.train['input_ids'], self.train['attention_mask'], self.train['labels'])
    train_data = DataLoader(dataset, sampler = RandomSampler(dataset), batch_size = self.batch_size)
    return train_data

  def val_dataloader(self):
    dataset = TensorDataset(self.validate['input_ids'], self.validate['attention_mask'], self.validate['labels'])
    val_data = DataLoader(dataset, batch_size = self.batch_size)
    return val_data

  def test_dataloader(self):
    dataset = TensorDataset(self.test['input_ids'], self.test['attention_mask'], self.test['labels'])
    test_data = DataLoader(dataset, batch_size = self.batch_size)
    return test_data

In [ ]:
def shift_tokens_right(input_ids, pad_token_id):
  """ Shift input ids one token to the right, and wrap the last non pad token (usually <eos>).
      This is taken directly from modeling_bart.py
  """
  prev_output_tokens = input_ids.clone()
  index_of_eos = (input_ids.ne(pad_token_id).sum(dim=1) - 1).unsqueeze(-1)
  prev_output_tokens[:, 0] = input_ids.gather(1, index_of_eos).squeeze()
  prev_output_tokens[:, 1:] = input_ids[:, :-1]
  return prev_output_tokens

def encode_sentences(tokenizer, source_sentences, target_sentences, max_length=1024, min_length = 512, pad_to_max_length=True, return_tensors="pt"):
  ''' Function that tokenizes a sentence
      Args: tokenizer - the BART tokenizer; source and target sentences are the source and target sentences
      Returns: Dictionary with keys: input_ids, attention_mask, target_ids
  '''

  input_ids = []
  attention_masks = []
  target_ids = []
  tokenized_sentences = {}

  for sentence in source_sentences:
    encoded_dict = tokenizer(
          sentence,
          max_length=max_length,
          padding="max_length" if pad_to_max_length else None,
          truncation=True,
          return_tensors=return_tensors,
          add_prefix_space = True
      )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

  input_ids = torch.cat(input_ids, dim = 0)
  attention_masks = torch.cat(attention_masks, dim = 0)

  for sentence in target_sentences:
    encoded_dict = tokenizer(
          sentence,
          max_length=min_length,
          padding="max_length" if pad_to_max_length else None,
          truncation=True,
          return_tensors=return_tensors,
          add_prefix_space = True
      )
    # Shift the target ids to the right
    # shifted_target_ids = shift_tokens_right(encoded_dict['input_ids'], tokenizer.pad_token_id)
    target_ids.append(encoded_dict['input_ids'])

  target_ids = torch.cat(target_ids, dim = 0)


  batch = {
      "input_ids": input_ids,
      "attention_mask": attention_masks,
      "labels": target_ids,
  }

  return batch

## Fine-Tuning Pegasus-X

In [ ]:
filepath = "/content/mcs.xlsx"
df = pd.read_excel(filepath,usecols='B,C')
#df = pd.read_excel(filename,index_col=0)
df.rename(columns = {'data':'source', 'summary':'target'}, inplace = True)

In [ ]:
df.head()

,source,target
0,Appeal No. LXVI of 1949. Appeal from the High ...,The charge created in respect of municipal pro...
1,"Dudhuria vs Commissioner of Income tax, Calcut...",The charge in respect of urban immoveable prop...
2,"Kania J., as he then was, said as follows : ""I...","The expression ""capital charge"" in s.9(1) (iv)..."


In [ ]:
# Loading Model and tokenizer
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, BartConfig

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large', add_prefix_space=True)

bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")

In [ ]:
# Add special tokens if required

new_tokens = ['<F>', '<RLC>', '<A>', '<S>', '<P>', '<R>', '<RPC>']

special_tokens_dict = {'additional_special_tokens': new_tokens}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
bart_model.resize_token_embeddings(len(tokenizer))

Embedding(50272, 1024)

In [ ]:
summary_data = SummaryDataModule(tokenizer, df, batch_size = 1)
model = LitModel(learning_rate = 2e-5, tokenizer = tokenizer, model = bart_model)

In [ ]:
# put in chatgpt for code implementation - "finetuning BART model with LitModel multi GPU implementation with tensorflow"
# for early stopping - "finetuning BART model with LitModel and adding a early stopping criterea with lightning module"

In [ ]:
# import pytorch_lightning as pl

trainer = pl.Trainer(accelerator="auto", # "accelarator='auto'" keyword which automatically uses all available GPUs
                     max_epochs = 3,
                     min_epochs = 2,
                     precision = 16)

INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, summary_data)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                         | Params
-------------------------------------------------------
0 | model | BartForConditionalGeneration | 406 M 
-------------------------------------------------------
406 M     Trainable params
0         Non-trainable params
406 M     Total params
1,625.194 Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


In [ ]:
#Save the model
trainer.save_checkpoint("output.ckpt")

## Evaluation (Extractive) of Pre-Trained Pegasus


In [ ]:
!pip install evaluate
!pip install bert_score

In [ ]:
import sys
sys.path.insert(0, '../')
import glob
import os
import json
import pandas as pd
import nltk
nltk.download('punkt')
from rouge_score import rouge_scorer
from rouge import Rouge
import math
import numpy as np
from tqdm import tqdm
from evaluate import load

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
ext_analysis = pd.read_csv('Extractive Test Dataset Analysis.csv')

In [ ]:
names, summ_word_count = list(ext_analysis['File']), list(ext_analysis['L = Average Summary Word Count'])

get_req_len = {}
for i in range(len(names)):
  get_req_len[names[i]] = summ_word_count[i]

In [ ]:
get_req_len

{'2012_S_270.txt': 1362,
 '2014_J_33.txt': 1007,
 '2014_R_41.txt': 1078,
 '2015_J_10.txt': 1126,
 '2015_S_368.txt': 1400}

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = 'google/pegasus-x-base'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

In [ ]:
def preprocess(text):
    input_ids = tokenizer.encode(text, truncation=True, return_tensors='pt')
    return input_ids

# Example usage
text = """Virat Kohli (Hindi pronunciation: [ʋɪˈɾɑːʈ ˈkoːɦli] (listen); born 5 November 1988) is an Indian international cricketer and the former captain of the Indian national cricket team who plays as a right-handed batsman for Royal Challengers Bangalore in the IPL and for Delhi in Indian domestic cricket. Widely regarded as one of the greatest batsmen of all time,[4] Kohli holds the records for scoring most runs in T20 internationals and in the IPL. In 2020, the International Cricket Council named him the male cricketer of the decade. Kohli has also contributed to India's successes, including winning the 2011 World Cup and the 2013 Champions trophy.

Born and raised in New Delhi, Kohli trained at the West Delhi Cricket Academy and started his youth career with the Delhi Under-15 team. He made his international debut in 2008 and quickly became a key player in the ODI team and later made his Test debut in 2011. In 2013, Kohli reached the number one spot in the ICC rankings for ODI batsmen for the first time. During 2014 T20 World Cup, he set a record for the most runs scored in the tournament. In 2018, he achieved yet another milestone, becoming the world's top-ranked Test batsman, making him the only Indian cricketer to hold the number one spot in all three formats of the game. His form continued in 2019, when he became the first player to score 20,000 international runs in a single decade. In 2021, Kohli made the decision to step down as the captain of the Indian national team for T20Is, following the T20 World Cup and in early 2022 he stepped down as the captain of the Test team as well.

He has received many accolades for his performances on the cricket field. He was recognized as the ICC ODI Player of the Year in 2012 and has won the Sir Garfield Sobers Trophy, given to the ICC Cricketer of the Year, on two occasions, in 2017 and 2018 respectively. Subsequently, Kohli also won ICC Test Player of the Year and ICC ODI Player of the Year awards in 2018, becoming the first player to win both awards in the same year. Additionally, he was named the Wisden Leading Cricketer in the World for three consecutive years, from 2016 to 2018. At the national level, Kohli was honoured with the Arjuna Award in 2013, the Padma Shri under the sports category in 2017 and the Khel Ratna award, India's highest sporting honour, in 2018."""
input_ids = preprocess(text)
print(len(text.split(' ')))

412


In [ ]:
model.eval()  # Set the model to evaluation mode

with torch.no_grad():
    output = model.generate(input_ids, num_beams=1, early_stopping=True)

summary = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"Summary: {summary}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (16384) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Summary: ,,, each a...


In [ ]:
txt = ""'Some people belive that the so called "face" on mars was created by life on mars. This is not the case. The face on Mars is a naturally occuring land form called a mesa. It was not created by aliens, and there is no consiracy to hide alien lifeforms on mars. There is no evidence that NASA has found that even suggests that this face was created by aliens.\n\nA mesa is a naturally occuring rock formation, that is found on Mars and Earth. This "face" on mars only looks like a face because humans tend to see faces wherever we look, humans are obviously extremely social, which is why our brain is designed to recognize faces.\n\nMany conspiracy theorists believe that NASA is hiding life on Mars from the rest of the world. These people would be very wrong. If NASA found life on Mars, then they would get millions of people\'s attention. NASA\'s budget would increase drasticly, which means that their workers would get paid more. There is no good reason that NASA would hide life on Mars from the rest of the world.\n\nSo, NASA is not hiding life on Mars from us, and they are not trying to trick us into thinking that the "face" on mars is just a mesa, because it actually is. NASA hiding life would be illogical, because if they found life on Mars, they would make a lot of money, and we all know that the people at NASA aren\'t illogical people.'""

In [ ]:
from transformers import AutoTokenizer, PegasusXForConditionalGeneration

model = PegasusXForConditionalGeneration.from_pretrained("google/pegasus-x-base")
tokenizer = AutoTokenizer.from_pretrained("google/pegasus-x-base")

ARTICLE_TO_SUMMARIZE = (
    "PG&E stated it scheduled the blackouts in response to forecasts for high winds "
    "amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were "
    "scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."
)
inputs = tokenizer(ARTICLE_TO_SUMMARIZE, max_length=1024, return_tensors="pt")  # `max_new_tokens` to control the maximum length of the generation.

# Generate Summary
summary_ids = model.generate(inputs["input_ids"])
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (16384) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


KeyboardInterrupt: ignored

In [ ]:
model_name = "google/pegasus-x-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
encoder_max_length = 1024*16
decoder_max_length = 1024

In [ ]:
led = AutoModelForSeq2SeqLM.from_pretrained(model_name, gradient_checkpointing=True, use_cache=False)
# set generate hyperparameters
led.config.num_beams = 2
led.config.max_length = decoder_max_length
led.config.min_length = 256
led.config.early_stopping = True
led.config.no_repeat_ngram_size = 4

In [ ]:
import torch
led.eval()

def generate_summary_gpu(input_text, l):
	inputs = tokenizer(
		input_text,
		padding="max_length",
		truncation=True,
		max_length=encoder_max_length,
		return_tensors="pt",
	)
	input_ids = inputs['input_ids']
	attention_mask = inputs['attention_mask']
	global_attention_mask = torch.zeros_like(attention_mask)
	# put global attention on <s> token
	global_attention_mask[:, 0] = 1
	l = max(l,256)
	l = min(l,1024)
	outputs = led.generate(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask, max_length = l, num_beams = 2,repetition_penalty = 2.5,early_stopping = True)
	preds = [tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_space=True) for gen_id in outputs]
	return "".join(preds)

In [ ]:
import torch
led.eval()

def generate_summary_gpu(input_text, l):
  summaries = []
  input_tokenized = tokenizer.encode(input_text, truncation=True, return_tensors='pt')
  summary_ids = led.generate(input_tokenized,
                                      length_penalty=0.01,
                                      min_length=l-5,
                                      max_length=l+5)
  output = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
  summaries.append(output)
  summaries = [sentence for sublist in summaries for sentence in sublist]
  return summaries

### Testing A1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def get_root_path():
    '''
    function to get root path of dataset
    change the path variable to the path of the dataset
    '''
    path = "/content/drive/MyDrive/ExtractiveDataset/test-data"  # use all 50 docs for testing if using abstractive dataset for training
    return path

def get_summary_data():
    '''
    function to get names, documents, and summaries
    change the path variable to the path of the dataset
    '''

    path = get_root_path() + '/judgement'
    all_files = glob.glob(path + "/*.txt")
    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f:
            p = filename.rfind("/")
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)
    path = get_root_path() + '/summary/A1'
    all_files = glob.glob(path + "/*.txt")
    data_summary = []
    for filename in all_files:
        with open(filename, 'r') as f:
            a = f.read()
            l = len(a)
            data_summary.append(a)

    return names, data_source, data_summary

def split_to_sentences(para):
    sents = nltk.sent_tokenize(para)   # returns list of sentences from para
    return sents

In [ ]:
names, data_source, data_summary = get_summary_data()
print(len(names))
print(len(data_source))
print(len(data_summary))


5
5
5


In [ ]:
# main loop to generate and save summaries of each document in the test dataset

result_rouge_list = []
result_bertscore_list = []

for i in tqdm(range(2)):  # range(len(data_source))
    name = names[i]
    doc = data_source[i]
    input_len = len(doc.split(" "))
    req_len = get_req_len[name]
    print(str(i) + ": " + name +  " - " + str(input_len) + " : " + str(req_len), end = " ,")

    extr_summ = generate_summary_gpu(doc,req_len)  # returns list of chunks summaries
    #print('--------------------------')

    if len(extr_summ.split(" ")) > req_len:
        extr_summ = extr_summ.split(" ")
        extr_summ = extr_summ[:req_len]
        extr_summ = " ".join(extr_summ)

    print(len(data_summary[i].split(' ')),len(extr_summ.split(' ')))

    rouge = Rouge()
    result_rouge_list.append(rouge.get_scores(extr_summ, data_summary[i], avg=True))
    bertscore = load("bertscore")
    result_bertscore_list.append(bertscore.compute(predictions=[extr_summ], references=[data_summary[i]], model_type="distilbert-base-uncased"))
    #print('---------------------------------------------------------------------------------------------------------------')

  0%|          | 0/2 [00:00<?, ?it/s]

0: 2012_S_270.txt - 4248 : 1362 ,

  0%|          | 0/2 [13:16<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
agg_rouge_score_1 = {'rouge-1':{'r':0,'p':0,'f':0},'rouge-2':{'r':0,'p':0,'f':0},
                   'rouge-l':{'r':0,'p':0,'f':0}}

for i in range(len(result_rouge_list)):
  for rouge_type in ['rouge-1','rouge-2','rouge-l']:
    for j in ['r','p','f']:
      agg_rouge_score_1[rouge_type][j]+=(result_rouge_list[i][rouge_type][j]/len(result_rouge_list))

print(agg_rouge_score_1)

{'rouge-1': {'r': 0.34919517102615694, 'p': 0.4248799292214358, 'f': 0.38269036953218116}, 'rouge-2': {'r': 0.17655071776181952, 'p': 0.19880821541420524, 'f': 0.1858949298444496}, 'rouge-l': {'r': 0.3137683242311009, 'p': 0.3802641557128412, 'f': 0.3432646344228572}}


In [ ]:
agg_bertscore_1 = {'precision':0, 'recall':0, 'f1':0}

for i in range(len(result_bertscore_list)):
  for metric_type in ['precision','recall','f1']:
    agg_bertscore_1[metric_type]+=((result_bertscore_list[i][metric_type][0])/len(result_bertscore_list))

print(agg_bertscore_1)

{'precision': 0.7688697874546051, 'recall': 0.7728851437568665, 'f1': 0.7708407342433929}


### Testing A2

In [ ]:
def get_root_path():
    '''
    function to get root path of dataset
    change the path variable to the path of the dataset
    '''
    path = "/content/drive/MyDrive/ExtractiveDataset/test-data"  # use all 50 docs for testing if using abstractive dataset for training
    return path

def get_summary_data():
    '''
    function to get names, documents, and summaries
    change the path variable to the path of the dataset
    '''

    path = get_root_path() + '/judgement'
    all_files = glob.glob(path + "/*.txt")
    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f:
            p = filename.rfind("/")
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)
    path = get_root_path() + '/summary/A2'
    all_files = glob.glob(path + "/*.txt")
    data_summary = []
    for filename in all_files:
        with open(filename, 'r') as f:
            a = f.read()
            l = len(a)
            data_summary.append(a)

    return names, data_source, data_summary

def split_to_sentences(para):
    sents = nltk.sent_tokenize(para)   # returns list of sentences from para
    return sents

In [ ]:
names, data_source, data_summary = get_summary_data()
print(len(names))
print(len(data_source))
print(len(data_summary))


5
5
5


In [ ]:
# main loop to generate and save summaries of each document in the test dataset

result_rouge_list = []
result_bertscore_list = []

for i in tqdm(range(len(data_source))):
    name = names[i]
    doc = data_source[i]
    input_len = len(doc.split(" "))
    req_len = get_req_len[name]
    print(str(i) + ": " + name +  " - " + str(input_len) + " : " + str(req_len), end = " ,")

    nested = nest_sentences(doc,512)  # returns a list of chunks with word count < 512
    p = float(req_len/input_len)  # p is the ratio of summary length to doc length
    print('Value of p for doc {} is {}'.format(i,p))

    extr_summ = generate_summary_gpu(nested,p)  # returns list of chunks summaries

    extr_summ = " ".join(extr_summ)

    if len(extr_summ.split(" ")) > req_len:
        extr_summ = extr_summ.split(" ")
        extr_summ = extr_summ[:req_len]
        extr_summ = " ".join(extr_summ)

    print(len(data_summary[i].split(' ')),len(extr_summ.split(' ')))

    rouge = Rouge()
    result_rouge_list.append(rouge.get_scores(extr_summ, data_summary[i], avg=True))
    bertscore = load("bertscore")
    result_bertscore_list.append(bertscore.compute(predictions=[extr_summ], references=[data_summary[i]], model_type="distilbert-base-uncased"))
    print('---------------------------------------------------------------------------------------------------------------')

  0%|          | 0/5 [00:00<?, ?it/s]

0: 2012_S_270.txt - 4248 : 1362 ,Value of p for doc 0 is 0.3206214689265537


In [ ]:
agg_rouge_score_2 = {'rouge-1':{'r':0,'p':0,'f':0},'rouge-2':{'r':0,'p':0,'f':0},
                   'rouge-l':{'r':0,'p':0,'f':0}}

for i in range(len(result_rouge_list)):
  for rouge_type in ['rouge-1','rouge-2','rouge-l']:
    for j in ['r','p','f']:
      agg_rouge_score_2[rouge_type][j]+=(result_rouge_list[i][rouge_type][j]/len(result_rouge_list))

print(agg_rouge_score_2)

{'rouge-1': {'r': 0.21187794059777082, 'p': 0.49926828337230633, 'f': 0.2935809851176972}, 'rouge-2': {'r': 0.07604486106863347, 'p': 0.20917546307120924, 'f': 0.1102924148470025}, 'rouge-l': {'r': 0.20117182810722975, 'p': 0.47589578811019456, 'f': 0.2791278810436789}}


In [ ]:
agg_bertscore_2 = {'precision':0, 'recall':0, 'f1':0}

for i in range(len(result_bertscore_list)):
  for metric_type in ['precision','recall','f1']:
    agg_bertscore_2[metric_type]+=((result_bertscore_list[i][metric_type][0])/len(result_bertscore_list))

print(agg_bertscore_2)

{'precision': 0.7443345665931702, 'recall': 0.751188361644745, 'f1': 0.7473442792892456}


###Average ROUGE score and BERTScores

In [ ]:
agg_rouge_score = {'rouge-1':{'r':0,'p':0,'f':0},'rouge-2':{'r':0,'p':0,'f':0},
                   'rouge-l':{'r':0,'p':0,'f':0}}

for rouge_type in ['rouge-1','rouge-2','rouge-l']:
  for j in ['r','p','f']:
    agg_rouge_score[rouge_type][j]=(agg_rouge_score_1[rouge_type][j]+agg_rouge_score_2[rouge_type][j])*0.5

print(agg_rouge_score)

In [ ]:
print('ROUGE Scores of A1 Golden summary:')
print(agg_rouge_score_1)
print()
print('ROUGE Scores of A2 Golden summary:')
print(agg_rouge_score_2)
print()
print('ROUGE Average ROUGE scores of both summaries:')
print(agg_rouge_score)

In [ ]:
agg_bertscore = {'precision':0, 'recall':0, 'f1':0}

for metric_type in ['precision','recall','f1']:
  agg_bertscore[metric_type]+=(agg_bertscore_1[metric_type]+agg_bertscore_2[metric_type])*0.5

print(agg_bertscore)

In [ ]:
print('BERTScores of A1 Golden summary:')
print(agg_bertscore_1)
print()
print('BERTScores of A2 Golden summary:')
print(agg_bertscore_2)
print()
print('Average BERTScores scores of both summaries:')
print(agg_bertscore)